In [1]:
import xarray as xr
import dscim
import yaml
from dscim.menu.simple_storage import Climate, EconVars
import pandas as pd
import numpy as np
from xarray.testing import assert_allclose
from xarray.testing import assert_equal
from itertools import product

import dask
import logging
import subprocess
from subprocess import CalledProcessError
from abc import ABC, abstractmethod
from dscim.descriptors import cachedproperty
from itertools import product
from dscim.menu.decorators import save
from dscim.menu.simple_storage import StackedDamages, EconVars
from dscim.utils.utils import (
    model_outputs,
    compute_damages,
    c_equivalence,
    power,
    quantile_weight_quantilereg,
    extrapolate,
)

In [12]:
# Config vs function params:
# Config should have all paths and parameters that are messy

# function params should be all of the simple parameters that catually want to be changed by EPA

def epa_scc(eta = 2.0,
            rho = 0.0,
            pulse_year = 2020,
            discount_type = "euler_ramsey",
            menu_option = "risk_aversion",
            gases = ['CO2_Fossil', 'CH4', 'N2O'],
            weitzman_parameters = [0.1, 0.5],
            fair_aggregation = ["ce", "mean", "gwr_mean", "median"],
            ssps = ['SSP2', 'SSP3', 'SSP4']):
    
    USER = "liruixue"
    master = f"/home/{USER}/repos/dscim-epa/damage_fun_runs/master-CAMEL_m1_c0.20.yaml"

    
    with open(master, "r") as stream:
        conf = yaml.safe_load(stream)
    
    conf["global_parameters"] = {'fair_aggregation': fair_aggregation,
     'subset_dict': {'ssp': ssps},
     'weitzman_parameter': weitzman_parameters,
     'save_files': []}

    conf['global_parameters']['fair_aggregation'] 
    save_path = "/home/liruixue/replication_newcode/"


    econ = EconVars(
        path_econ="/shares/gcp/integration_replication/inputs/econ/rff_global_socioeconomics.nc4"
    )

    MENU_OPTIONS = {
        "adding_up": dscim.menu.baseline.Baseline,
        "risk_aversion": dscim.menu.risk_aversion.RiskAversionRecipe,
        "equity": dscim.menu.equity.EquityRecipe,
    }
    add_kwargs = {
        "econ_vars": econ,
        "climate_vars": Climate(**conf["rff_climate"], pulse_year=pulse_year),
        "formula": 'damages ~ -1 + anomaly + np.power(anomaly, 2) + gmsl + np.power(gmsl, 2)',
        "discounting_type": discount_type,
        "sector": "CAMEL_m1_c0.20",
        "ce_path": None,
        "save_path": save_path,
        "eta": eta,
        "rho": rho,
        "damage_function_path": "/shares/gcp/integration_replication/results/rff/CAMEL_m1_c0.20/2020",
        "ecs_mask_path": None,
        "ecs_mask_name": None,
    }

    kwargs = conf["global_parameters"].copy()
    for k, v in add_kwargs.items():
        assert (
            k not in kwargs.keys()
        ), f"{k} already set in config. Please check `global_parameters`."
        kwargs.update({k: v})

    menu_item = MENU_OPTIONS[menu_option](**kwargs)
    menu_item.order_plate("scc")
    uncollapsed_scc = menu_item.uncollapsed_sccs
    
    return(menu_item)
    
# This represents the full gamut of scc runs when run default
def epa_sccs(etas_rhos = [[2.0, 0.0],
               [1.016010255, 9.149608e-05],
               [1.244459066, 0.00197263997],
               [1.421158116, 0.00461878399],
               [1.567899395, 0.00770271076]],
             risk_combos = [['risk_aversion', 'euler_ramsey'],
                           ['adding_up','euler_ramsey']],
             pulse_year = 2020,
             gases = ['CO2_Fossil', 'CH4', 'N2O'],
             weitzman_parameters = [0.1, 0.5],
             fair_aggregation = [None],
             ssps = ['SSP2', 'SSP3', 'SSP4']):

    coords = {"eta_rhos": [str(i) + "_" + str(j) for i,j in etas_rhos],
          "discount_type": [j for i,j in risk_combos],
          "runid": range(1,10001),
          "gas": ['CO2_Fossil', 'CH4', 'N2O'],
          "simulation": [1],
          "weitzman_parameter":['0.1','0.5'],
          "fair_aggregation": ['uncollapsed'],
          "menu_option": [i for i,j in risk_combos]
         }
    all_arrays_uscc = []
    all_arrays_gcnp = []
    for i, j in product(etas_rhos, risk_combos):
        eta = i[0]
        rho = i[1]
        discount_type= j[1]
        menu_option = j[0]
        df_single = epa_scc(discount_type = discount_type,
                            menu_option = menu_option,
                            eta = eta,
                            rho = rho,
                            pulse_year = 2020,
                            gases = gases,
                            weitzman_parameters = weitzman_parameters,
                            fair_aggregation = fair_aggregation,
                            ssps = ssps)
        
        df_scc = df_single.uncollapsed_sccs.assign_coords(eta_rhos =  str(eta) + "_" + str(rho), menu_option = menu_option)
        df_scc_expanded = df_scc.expand_dims(['eta_rhos','menu_option'])
        df_scc_expanded.name = "uncollapsed_sccs"
        df_scc_expanded = df_scc_expanded.to_dataset()
        all_arrays_uscc = all_arrays_uscc + [df_scc_expanded]
        
        df_gcnp = df_single.global_consumption_no_pulse.assign_coords(eta_rhos =  str(eta) + "_" + str(rho), menu_option = menu_option)
        df_gcnp_expanded = df_gcnp.expand_dims(['eta_rhos','menu_option'])
        df_gcnp_expanded.name = "global_consumption_no_pulse"
        df_gcnp_expanded = df_gcnp_expanded.to_dataset()
        all_arrays_gcnp = all_arrays_gcnp + [df_gcnp_expanded]
        
    df_full_scc = xr.combine_by_coords(all_arrays_uscc)
    df_full_scc.to_netcdf("/home/liruixue/replication_newcode/full_order_uncollapsed_sccs.nc4")    
    
    df_full_gcnp = xr.combine_by_coords(all_arrays_gcnp)
    df_full_gcnp.to_netcdf("/home/liruixue/replication_newcode/full_order_global_consumption_no_pulse.nc4")    




In [13]:
etas_rhos = [[2.0, 0.0],
             [1.016010255, 9.149608e-05],
             [1.244459066, 0.00197263997],
             [1.421158116, 0.00461878399],
             [1.567899395, 0.00770271076]]
#etas_rhos = [[2.0,0.0]]
risk_combos = [['risk_aversion', 'euler_ramsey'],
               ['risk_aversion','constant']]
pulse_year = 2020
gases = ['CO2_Fossil', 'CH4', 'N2O']
weitzman_parameters = [0.1, 0.5]
fair_aggregation = ["mean"]
ssps = ['SSP2', 'SSP3', 'SSP4']
test = epa_sccs(etas_rhos,
                risk_combos,
                pulse_year=2020,
                fair_aggregation = fair_aggregation)    



 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 2.0
        rho: 0.0
        
Processing damage functions ...
Skipped saving damage_function_coefficients.
Subsetting on {} keys.
Global consumption found up to 2300.
Skipped saving global_consumption.
Skipped saving damage_function_coefficients.
Skipped saving global_consumption_no_pulse.
Processing SCC calculation ...
Subsetting on {} keys.
Skipped saving discount_factors.
Skipped saving damage_function_coefficients.
Skipped saving global_consumption_pulse.
Skipped saving marginal_damages.
Skipped saving scc.
Skipped saving uncollapsed_sccs.
Skipped saving uncollapsed_marginal_damages.
Skipped saving uncollapsed_discount_factors.
Results available: /home/liruixue/replication_newcode/

 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: constant
        eta: 2.0
        rho: 0.0
        
Processing damage functions ...
Skipp

In [14]:
test1 = xr.open_dataset("/home/liruixue/replication_newcode/full_order_uncollapsed_sccs.nc4")
test1

<xarray.Dataset>
Dimensions:             (fair_aggregation: 1, discount_type: 2, runid: 10000, gas: 3, simulation: 1, weitzman_parameter: 2, discrate: 6, eta_rhos: 5, menu_option: 1)
Coordinates:
  * fair_aggregation    (fair_aggregation) object 'uncollapsed'
  * discount_type       (discount_type) object 'constant' 'euler_ramsey'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * simulation          (simulation) int64 1
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.5'
  * discrate            (discrate) float64 0.01 0.015 0.02 0.025 0.03 0.05
  * eta_rhos            (eta_rhos) object '1.016010255_9.149608e-05' ... '2.0...
  * menu_option         (menu_option) object 'risk_aversion'
Data variables:
    uncollapsed_sccs    (eta_rhos, menu_option, discrate, fair_aggregation, weitzman_parameter, discount_type, runid, gas, simulation) float64 ...

In [15]:
test2 = xr.open_dataset("/home/liruixue/replication_newcode/full_order_global_consumption_no_pulse.nc4")
test2

<xarray.Dataset>
Dimensions:                      (discount_type: 2, year: 281, runid: 10000, gas: 3, simulation: 1, weitzman_parameter: 2, eta_rhos: 5, menu_option: 1)
Coordinates:
  * discount_type                (discount_type) object 'constant' 'euler_ram...
  * year                         (year) int64 2020 2021 2022 ... 2298 2299 2300
  * runid                        (runid) int64 1 2 3 4 ... 9997 9998 9999 10000
  * gas                          (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * simulation                   (simulation) int64 1
  * weitzman_parameter           (weitzman_parameter) object '0.1' '0.5'
  * eta_rhos                     (eta_rhos) object '1.016010255_9.149608e-05'...
  * menu_option                  (menu_option) object 'risk_aversion'
Data variables:
    global_consumption_no_pulse  (eta_rhos, menu_option, weitzman_parameter, discount_type, runid, year, gas, simulation) float64 ...